In [1]:
import warnings
import os
import sys

warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))
sys.path.append(parent_dir)

In [1]:
from src.main.models.bidirLSMT import BidirectionalLSTM
from src.main.utilities.utils import clean_text, stop_words_removal, stemming, tfidf_vectorizer, get_dataset
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabriziodecastelli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# define model and pipeline
bidirLSTM = BidirectionalLSTM(vocab_size=10000, embedding_dim=100, max_sequence_length=75, lstm_units=64)
bidirLSTM.set_pipeline([clean_text, stop_words_removal])

# import dataset and apply pipeline
inputs, targets = get_dataset(one_hot=True)

inputs = bidirLSTM.run_pipeline(inputs, save=False)

# stratified train test split
x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state=42, stratify=targets)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabriziodecastelli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabriziodecastelli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabriziodecastelli/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabriziodecastelli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabriziodecastelli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabriziodecastelli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fa

Pipeline execution time: 0:00:02.514538


In [13]:
print(inputs[0])
print(targets.head(1))

23 funniest tweets cats dogs week sept 1723 dog dont understand could eaten
   Entertainment   Life  Politics  Sports  Voices
2           True  False     False   False   False


In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def tokenize(sequences):
    # Tokenization
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sequences)
    sequences_as_integers = tokenizer.texts_to_sequences(sequences)

    # Padding
    max_sequence_length = 100  # Choose an appropriate value based on your data
    return pad_sequences(sequences_as_integers, maxlen=max_sequence_length)

In [15]:
x_train = tokenize(x_train)
x_test = tokenize(x_test)

In [16]:
bidirLSTM.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, None, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ (None, None, 128)      │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_7 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,183,941 (4.52 MB)

 Trainable params: 1,183,941 (4.52 MB)

 Non-trainable params: 0 (0.00 B)

None


In [17]:
#bidirLSTM.fit(x_train, y_train)
bidirLSTM.bidirLSTM.fit(x_train, y_train, batch_size=256, epochs=3, validation_data=(x_test, y_test))

Epoch 1/3
339/339 ━━━━━━━━━━━━━━━━━━━━ 116s 336ms/step - accuracy: 0.3824 - loss: 1.4032 - val_accuracy: 0.3904 - val_loss: 1.3911
Epoch 2/3
339/339 ━━━━━━━━━━━━━━━━━━━━ 121s 357ms/step - accuracy: 0.4028 - loss: 1.3739 - val_accuracy: 0.3774 - val_loss: 1.4045
Epoch 3/3
339/339 ━━━━━━━━━━━━━━━━━━━━ 1478s 4s/step - accuracy: 0.4647 - loss: 1.2633 - val_accuracy: 0.3399 - val_loss: 1.4886


In [ ]:
#model predictions

y_pred = bidirLSTM.predict(x_test)

In [14]:
bidirLSTM.bidirLSTM.evaluate(x_test, y_test)

677/677 ━━━━━━━━━━━━━━━━━━━━ 17s 25ms/step - accuracy: 0.3556 - loss: 2.5909


[2.5806286334991455, 0.3578621447086334]

In [13]:
results = bidirLSTM.compute_performance(y_test, y_pred)
bidirLSTM.print_performance(results)


677/677 ━━━━━━━━━━━━━━━━━━━━ 17s 25ms/step


ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets